## Reasoning trace experiments:
- Pick 100 wrong questions

- Questions to study:
    1. Conceptual errors: Did the VLM make any conceptual errors? (Inability to use the correct concepts required for a certain question)
    2. Grounding errors: Did the VLM model the question appropriately? (Utilizing the correct formulas based on the concepts, appropriate mathematical modeling, correct equation setups, etc.)
    3. Computation errors: Is the algebraic manipulation and arithmetic correct?
    4. Instruction FOllowing errors

## Selecting 100 wrong responses at random from OpenAI o3, InternVL3 8B, and Gemini 2.5 Pro

In [ ]:
import json
import random
import pandas as pd
from pathlib import Path

def load_json_file(file_path):
    """Load JSON file and return the results list"""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        return data.get('results', [])
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return []

def filter_incorrect_responses(results, model_name):
    """Filter to only include incorrect responses (is_correct = False)"""
    incorrect = []
    for result in results:
        if result.get('is_correct', True) == False:  # Only wrong answers
            incorrect.append(result)
    
    print(f"{model_name}: {len(results)} total responses, {len(incorrect)} incorrect responses")
    return incorrect

def filter_o3_responses(results):
    """Filter O3 responses to only include those >70 characters"""
    filtered = []
    for result in results:
        full_response = result.get('full_response', '')
        if len(full_response) > 70:
            filtered.append(result)
    
    print(f"O3: After length filter: {len(filtered)} responses >70 chars")
    return filtered

def select_random_questions(file_paths, questions_per_model=33):
    """
    Select random questions from each model
    
    Args:
        file_paths: Dictionary with model names as keys and file paths as values
        questions_per_model: Number of questions to select per model (default: 33)
    """
    
    all_selected = []
    selection_summary = {}
    
    for model_name, file_path in file_paths.items():
        print(f"\nProcessing {model_name}...")
        
        # Load results
        results = load_json_file(file_path)
        
        if not results:
            print(f"No results found for {model_name}")
            continue
        
        # Filter to only incorrect responses first
        results = filter_incorrect_responses(results, model_name)
        
        if not results:
            print(f"No incorrect responses found for {model_name}")
            continue
        
        # Apply O3 length filter if needed (after incorrect filter)
        if model_name.lower() == 'o3':
            results = filter_o3_responses(results)
        
        # Random selection
        if len(results) < questions_per_model:
            print(f"Warning: {model_name} has only {len(results)} questions, selecting all")
            selected = results
        else:
            selected = random.sample(results, questions_per_model)
        
        # Add model identifier to each selected question
        for question in selected:
            question['model_name'] = model_name
            question['source_file'] = file_path
        
        all_selected.extend(selected)
        selection_summary[model_name] = {
            'total_available': len(load_json_file(file_path)),  # Original total
            'incorrect_available': len(results),  # After filtering for incorrect
            'selected': len(selected),
            'file_path': file_path
        }
        
        print(f"Selected {len(selected)} questions from {model_name}")
    
    return all_selected, selection_summary

def save_selected_questions(selected_questions, output_file='selected_incorrect_questions_for_manual_review.json'):
    """Save selected questions to JSON file"""
    
    # Create summary data
    summary_data = {
        'selection_info': {
            'total_selected': len(selected_questions),
            'selection_date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
            'selection_method': 'random_sampling_incorrect_only',
            'filter_criteria': 'is_correct == False'
        },
        'selected_questions': selected_questions
    }
    
    # Save to file
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(summary_data, f, indent=2, ensure_ascii=False)
    
    print(f"\nSelected questions saved to: {output_file}")
    return output_file

def create_summary_report(selection_summary, selected_questions):
    """Create a summary report of the selection"""
    
    print("\n" + "="*60)
    print("SELECTION SUMMARY REPORT")
    print("="*60)
    
    # Overall summary
    total_selected = len(selected_questions)
    print(f"Total questions selected: {total_selected}")
    
    # Per-model breakdown
    print(f"\nPer-model breakdown:")
    for model, info in selection_summary.items():
        print(f"  {model}:")
        print(f"    - Total responses: {info['total_available']}")
        print(f"    - Incorrect responses: {info['incorrect_available']}")
        print(f"    - Selected for review: {info['selected']}")
        print(f"    - File: {Path(info['file_path']).name}")
    
    # Question type distribution
    print(f"\nQuestion type distribution in selected set:")
    question_types = {}
    subjects = {}
    languages = {}
    
    for q in selected_questions:
        # Question type
        qtype = q.get('question_type', 'Unknown')
        question_types[qtype] = question_types.get(qtype, 0) + 1
        
        # Subject
        subject = q.get('subject', 'Unknown')
        subjects[subject] = subjects.get(subject, 0) + 1
        
        # Language
        language = q.get('language', 'Unknown')
        languages[language] = languages.get(language, 0) + 1
    
    print(f"  Question Types: {dict(sorted(question_types.items()))}")
    print(f"  Subjects: {dict(sorted(subjects.items()))}")
    print(f"  Languages: {dict(sorted(languages.items()))}")

# Set random seed for reproducibility
random.seed(42)

# Define file paths
file_paths = {
    'gemini25': r'gemini25_evaluation_results\gemini25_run_04_20250724_202554.json',
    'o3': r'o3_evaluation_results\o3_run_01_partial_20250719_092641.json',
    'internvl3_8b': r'internvl3_8b_evaluation_results\internvl3_8b_run_06_20250725_201612.json'
}

questions_per_model = 33  # 33 + 33 + 33 = 99

print("Starting random question selection from INCORRECT responses only...")
print(f"Target: ~{questions_per_model} incorrect questions per model")

# Select questions
selected_questions, selection_summary = select_random_questions(
    file_paths, questions_per_model
)

# Adjust if we need exactly 100
if len(selected_questions) > 100:
    selected_questions = random.sample(selected_questions, 100)
    print(f"\nAdjusted to exactly 100 questions")

# Save results
output_file = save_selected_questions(selected_questions)

# Create summary report
create_summary_report(selection_summary, selected_questions)

print(f"\n" + "="*60)
print("SELECTION COMPLETE!")
print(f"Output file: {output_file}")
print("="*60)